In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
fablib = fablib_manager()

In [2]:
import mflib 
print(f"MFLib version  {mflib.__version__} " )

from mflib.mflib import MFLib

MFLib version  1.0.4 


In [3]:
slice_name = "kalhan-cse-534-nat-2-servers-minimal-servers"

In [ ]:
#slice = fablib.new_slice(name=slice_name)

In [4]:
# # retrieve an existing slice
slice = fablib.get_slice(name=slice_name)

In [ ]:
#slice.delete()

In [5]:
slice.get_state()

'StableOK'

In [ ]:
# from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
# fablib = fablib_manager()
# slice = fablib.get_slice(name=slice_name)
# slice.delete()

In [ ]:
fablib.list_sites(latlon=False)

In [ ]:
site1='MICH'
site2='UTAH'
site3='NCSA'
site4 = 'STAR' # todo change topology and deploy switch at star. , USE STAR TOPOLOGY FOR DEPLOYONG THE SWITCH

print (f'The selected sites are {site1}, {site2}, {site3} {site4}') 

In [ ]:
client = slice.add_node(name="client", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server1 = slice.add_node(name="server1", 
                      site=site2, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')
server2 = slice.add_node(name="server2", 
                      site=site3, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')


In [ ]:
client_iface = client.add_component(model='NIC_Basic').get_interfaces()[0]
server1_iface = server1.add_component(model='NIC_Basic').get_interfaces()[0]
server2_iface = server2.add_component(model='NIC_Basic').get_interfaces()[0]


In [ ]:
# Add a node for P4 switch
switch = slice.add_node(name="switch", 
                      site=site4, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

In [ ]:
switch_client_iface = switch.add_component(model='NIC_Basic', name='client_nic').get_interfaces()[0]
switch_server1_iface = switch.add_component(model='NIC_Basic', name='server1_nic').get_interfaces()[0]
switch_server2_iface = switch.add_component(model='NIC_Basic', name='server2_nic').get_interfaces()[0]


In [ ]:
switch_client_net = slice.add_l2network(name='switch_client_net', interfaces=[client_iface, switch_client_iface])
switch_server1_net = slice.add_l2network(name='switch_server1_net', interfaces=[switch_server1_iface, server1_iface])
switch_server2_net = slice.add_l2network(name='switch_server2_net', interfaces=[switch_server2_iface, server2_iface])

In [ ]:

MFLib.addMeasNode(slice, disk=100)
print("Done")

In [ ]:
slice.submit()

In [32]:
for iface in slice.get_interfaces():
    print(iface)

---------------  ---------------------
Name             switch-server1_nic-p1
Network          switch_server1_net
Bandwidth        100
Mode             config
VLAN
MAC              1A:AC:EC:5E:AE:C8
Physical Device  enp7s0
Device           enp7s0
Address
Numa Node        6
---------------  ---------------------
---------------  --------------------
Name             switch-client_nic-p1
Network          switch_client_net
Bandwidth        100
Mode             config
VLAN
MAC              1A:B9:CE:DD:FF:7C
Physical Device  enp8s0
Device           enp8s0
Address
Numa Node        6
---------------  --------------------
---------------  ------------------------------
Name             switch-meas_nic_switch_STAR-p1
Network          l3_meas_net_STAR
Bandwidth        100
Mode             config
VLAN
MAC              22:6B:5E:2D:15:BF
Physical Device  enp9s0
Device           enp9s0
Address
Numa Node        6
---------------  ------------------------------
---------------  ------------------
Name

#  NETWORK services


In [38]:
for nw_service in slice.get_network_services():
    print(nw_service)

-------  ------------------------------------
ID       67232fdd-f859-450c-8ad0-bce05d493505
Name     switch_client_net
Layer    L2
Type     L2STS
Site
Gateway
Subnet
State    Active
Error
-------  ------------------------------------
-------  ------------------------------------
ID       09d08bf9-7c1a-4d16-aada-0175bc0c627b
Name     switch_server1_net
Layer    L2
Type     L2STS
Site
Gateway
Subnet
State    Active
Error
-------  ------------------------------------
-------  ------------------------------------
ID       1cf5defb-d984-4e0d-8446-62649c422982
Name     switch_server2_net
Layer    L2
Type     L2STS
Site
Gateway
Subnet
State    Active
Error
-------  ------------------------------------
-------  ------------------------------------
ID       16692911-bb4d-41f1-b3ab-b2cd5cf3609e
Name     l3_meas_net_MICH
Layer    L3
Type     FABNetv4
Site     MICH
Gateway  10.131.6.1
Subnet   10.131.6.0/24
State    Active
Error
-------  ------------------------------------
-------  --------------

## NODE INFORMATION

In [33]:
for node in slice.get_nodes():
    print(node)

-----------------  -----------------------------------------------------------------------------------------------------------------------------------------
ID                 7b8df70a-dd31-4828-ae7d-9acddcc80667
Name               switch
Cores              32
RAM                16
Disk               100
Image              default_ubuntu_20
Image Type         qcow2
Host               star-w2.fabric-testbed.net
Site               STAR
Management IP      2001:400:a100:3030:f816:3eff:fef5:4ece
Reservation State  Active
Error Message
SSH Command        ssh -i {{ _self_.private_ssh_key_file }} -F /home/fabric/work/fabric_config/ssh_config {{ _self_.username }}@{{ _self_.management_ip }}"s
-----------------  -----------------------------------------------------------------------------------------------------------------------------------------
-----------------  -----------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

#Set end host to now plus 1 day
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
### installing the switch normal, dont need incase of high performance switch



switch = slice.get_node(name="switch")     
switch.upload_file('install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

In [13]:
#The net-tools package will be installed on the switch, server1 and server2 nodes. This package will allow us to use the ifconfig and the arp commands

client = slice.get_node(name="client")
server1 = slice.get_node(name="server1")
server2 = slice.get_node(name="server2")
switch = slice.get_node(name="switch")
meas_node = slice.get_node(name= "meas-node")


stdout, stderr = client.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = meas_node.execute(f'sudo apt-get install -y net-tools', quiet=True)

### add l2 networj b/w switch and meas node

In [ ]:
switch_meas_iface = switch.add_component(model='NIC_Basic', name='meas_nic').get_interfaces()[0]
meas_iface = meas_node.add_component(model='NIC_Basic', name='meas_nic').get_interfaces()[0]
switch_meas_net = slice.add_l2network(name='switch_meas_net', interfaces=[switch_meas_iface, meas_iface])

In [39]:
slice.submit()


Retry: 1, Time: 352 sec


ID,9465d01f-d56a-44d4-bd36-66e5f7565a1a
Name,kalhan-cse-534-nat-2-servers-minimal-servers
Lease Expiration (UTC),2024-04-27 22:07:29 +0000
Lease Start (UTC),2024-04-20 19:17:02 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
a6a80240-264a-4fe7-9856-387dd1c7ec2b,client,4,8,100,default_ubuntu_20,qcow2,mich-w1.fabric-testbed.net,MICH,ubuntu,2607:f018:110:11:f816:3eff:fe18:3a36,Active,,"ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2607:f018:110:11:f816:3eff:fe18:3a36""s",/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
753a6b43-e9fc-48ec-b866-09510c1a58de,meas-node,4,16,100,default_ubuntu_20,qcow2,edc-w1.fabric-testbed.net,EDC,ubuntu,2620:0:c80:1003:f816:3eff:fe38:7a81,Active,,"ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1003:f816:3eff:fe38:7a81""s",/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
3dd6ebf1-d2b5-435a-8538-b26a25389ba9,server1,4,8,100,default_ubuntu_20,qcow2,utah-w5.fabric-testbed.net,UTAH,ubuntu,2001:1948:417:7:f816:3eff:fe43:8575,Active,,"ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:1948:417:7:f816:3eff:fe43:8575""s",/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
232bb484-f865-4052-aeb4-a00566d70ecb,server2,4,8,100,default_ubuntu_20,qcow2,ncsa-w2.fabric-testbed.net,NCSA,ubuntu,2620:0:c80:1001:f816:3eff:fe74:90b4,Active,,"ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:c80:1001:f816:3eff:fe74:90b4""s",/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7b8df70a-dd31-4828-ae7d-9acddcc80667,switch,32,16,100,default_ubuntu_20,qcow2,star-w2.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fef5:4ece,Active,,"ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fef5:4ece""s",/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
bfe50ac3-1b5a-47c2-b41d-6778d96f389f,l3_meas_net_EDC,L3,FABNetv4,EDC,10.132.147.0/24,10.132.147.1,Active,
16692911-bb4d-41f1-b3ab-b2cd5cf3609e,l3_meas_net_MICH,L3,FABNetv4,MICH,10.131.6.0/24,10.131.6.1,Active,
bf9bd777-cf71-4027-abe6-54276240a912,l3_meas_net_NCSA,L3,FABNetv4,NCSA,10.132.146.0/24,10.132.146.1,Active,
dca4b8a5-ee22-45d2-84d3-865bfc4bb6f7,l3_meas_net_STAR,L3,FABNetv4,STAR,10.129.135.0/24,10.129.135.1,Active,
96a88c19-448b-4ade-b07e-ab335ecb6b35,l3_meas_net_UTAH,L3,FABNetv4,UTAH,10.132.5.0/24,10.132.5.1,Active,
67232fdd-f859-450c-8ad0-bce05d493505,switch_client_net,L2,L2STS,None,None,None,Active,
197c8ecc-ffab-4445-8f4e-f93384d8a7e1,switch_meas_net,L2,L2STS,None,None,None,Active,
09d08bf9-7c1a-4d16-aada-0175bc0c627b,switch_server1_net,L2,L2STS,None,None,None,Active,
1cf5defb-d984-4e0d-8446-62649c422982,switch_server2_net,L2,L2STS,None,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
switch-client_nic-p1,p1,switch,switch_client_net,100,config,,1A:B9:CE:DD:FF:7C,enp8s0,enp8s0,None,6
switch-meas_nic_switch_STAR-p1,p1,switch,l3_meas_net_STAR,100,config,,22:6B:5E:2D:15:BF,enp9s0,enp9s0,None,6
switch-meas_nic-p1,p1,switch,switch_meas_net,100,config,,1A:CE:5E:50:C9:8E,enp11s0,enp11s0,None,6
switch-server1_nic-p1,p1,switch,switch_server1_net,100,config,,1A:AC:EC:5E:AE:C8,enp7s0,enp7s0,None,6
switch-server2_nic-p1,p1,switch,switch_server2_net,100,config,,22:BE:32:75:80:69,enp10s0,enp10s0,None,6
meas-node-meas_nic_meas-node_EDC-p1,p1,meas-node,l3_meas_net_EDC,100,config,,06:A4:94:C1:58:26,enp6s0,enp6s0,None,6
meas-node-meas_nic-p1,p1,meas-node,switch_meas_net,100,config,,0A:EC:DF:53:B2:7E,enp7s0,enp7s0,None,6
client-meas_nic_client_MICH-p1,p1,client,l3_meas_net_MICH,100,config,,16:00:22:5E:58:FC,enp8s0,enp8s0,None,6
client-None-p1,p1,client,switch_client_net,100,config,,0E:06:73:C1:D9:AE,enp7s0,enp7s0,None,6
server1-meas_nic_server1_UTAH-p1,p1,server1,l3_meas_net_UTAH,100,config,,0A:5C:5D:3E:8D:5D,enp7s0,enp7s0,None,4



Time to print interfaces 373 seconds


'9465d01f-d56a-44d4-bd36-66e5f7565a1a'

## EXECUTE BELOW FOR QUILY ACCESSING NODES

In [5]:
client = slice.get_node(name="client")
server1 = slice.get_node(name="server1")
switch = slice.get_node(name="switch")
server2 = slice.get_node(name="server2")
meas_node = slice.get_node(name= "meas-node")

In [6]:
client_iface = client.get_interface(network_name='switch_client_net') 
client_iface_name = client_iface.get_device_name()
print(f'client_iface_name: {client_iface_name}')

switch_client_iface = switch.get_interface(network_name='switch_client_net') 
switch_client_iface_name = switch_client_iface.get_device_name()
print(f'switch_client_iface_name: {switch_client_iface_name}')

switch_s1_iface = switch.get_interface(network_name='switch_server1_net') 
switch_s1_iface_name = switch_s1_iface.get_device_name()
print(f'switch_s1_iface_name: {switch_s1_iface_name}')

switch_s2_iface = switch.get_interface(network_name='switch_server2_net') 
switch_s2_iface_name = switch_s2_iface.get_device_name()
print(f'switch_s2_iface_name: {switch_s2_iface_name}')


switch_meas_iface = switch.get_interface(network_name='switch_meas_net') 
switch_meas_iface_name = switch_meas_iface.get_device_name()
print(f'switch_meas_iface_name: {switch_meas_iface_name}')

server1_iface = server1.get_interface(network_name='switch_server1_net') 
server1_iface_name = server1_iface.get_device_name()
print(f'server1_iface_name: {server1_iface_name}')

server2_iface = server2.get_interface(network_name='switch_server2_net') 
server2_iface_name = server2_iface.get_device_name()
print(f'server2_iface_name: {server2_iface_name}')

meas_iface = meas_node.get_interface(network_name='switch_meas_net') 
meas_iface_name = meas_iface.get_device_name()
print(f'meas_iface_name: {meas_iface_name}')

client_iface_name: enp7s0
switch_client_iface_name: enp8s0
switch_s1_iface_name: enp7s0
switch_s2_iface_name: enp10s0
switch_meas_iface_name: enp11s0
server1_iface_name: enp8s0
server2_iface_name: enp8s0
meas_iface_name: enp7s0


In [10]:
### turning all the ifaces up

stdout, stderr = client.execute(f'sudo ip link set dev {client_iface_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_client_iface_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_s1_iface_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_s2_iface_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_meas_iface_name} up', quiet=True)
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface_name} up', quiet=True)
stdout, stderr = meas_node.execute(f'sudo ip link set dev {meas_iface_name} up', quiet=True)

In [11]:
client_iface_MAC = '00:00:00:00:00:01'
switch_client_iface_MAC = '00:00:00:00:00:02'
switch_s1_iface_MAC = '00:00:00:00:00:03'
server1_iface_MAC = '00:00:00:00:00:04'
switch_s2_iface_MAC = '00:00:00:00:00:05'
server2_iface_MAC = '00:00:00:00:00:06'
meas_iface_MAC = '00:00:00:00:00:07'
switch_meas_iface_MAC = '00:00:00:00:00:08'


In [15]:

## configuring ip and mac b/w client_iface and switch_iface1

client_switch_subnet = "192.168.1.0/24"
client_iface_ip = '192.168.1.10/24'
switch_client_iface_ip = '192.168.1.1/24'

stdout, stderr = client.execute(f'sudo ifconfig {client_iface_name} {client_iface_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_client_iface_name} {switch_client_iface_ip}')

stdout, stderr = client.execute(f'sudo ifconfig {client_iface_name} hw ether {client_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_client_iface_name} hw ether {switch_client_iface_MAC}')

In [16]:

## configuring ip and mac b/w server1_iface and switch_s1_iface

server1_switch_subnet = "192.168.2.0/24"
server1_iface_ip = '192.168.2.10/24'
switch_s1_iface_ip = '192.168.2.1/24'

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_iface_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_s1_iface_name} {switch_s1_iface_ip}')

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_s1_iface_name} hw ether {switch_s1_iface_MAC}')

In [17]:

## configuring ip and mac b/w server2_iface and switch_s2_iface

server2_switch_subnet = "192.168.3.0/24"
server2_iface_ip = '192.168.3.10/24'
switch_s2_iface_ip = '192.168.3.1/24'

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} {server2_iface_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_s2_iface_name} {switch_s2_iface_ip}')

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} hw ether {server2_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_s2_iface_name} hw ether {switch_s2_iface_MAC}')

In [18]:

## configuring ip and mac b/w meas_iface and switch_meas_iface

meas_switch_subnet = "192.168.4.0/24"
meas_iface_ip = '192.168.4.10/24'
switch_meas_iface_ip = '192.168.4.1/24'

stdout, stderr = meas_node.execute(f'sudo ifconfig {meas_iface_name} {meas_iface_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_meas_iface_name} {switch_meas_iface_ip}')

stdout, stderr = meas_node.execute(f'sudo ifconfig {meas_iface_name} hw ether {meas_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_meas_iface_name} hw ether {switch_meas_iface_MAC}')

In [19]:

## enable ip forwarding on switch
command = 'sudo sysctl -w net.ipv4.ip_forward=1' 
stdout, stderr = switch.execute(command, quiet=True)

In [20]:
## In this step, we are deleting the routes on the switch's routing table in Linux. By deleting the routes, the packets will go through the BMv2 switch instead of being forwarded by the kernel

stdout, stderr = switch.execute(f'sudo ip route del {client_switch_subnet}', quiet=True)
stdout, stderr = switch.execute(f'sudo ip route del {server1_switch_subnet}', quiet=True)
stdout, stderr = switch.execute(f'sudo ip route del {server2_switch_subnet}', quiet=True)

# dont delete this route for meas_node

In [21]:
### configure static routing


switch_client_gw = switch_client_iface_ip.split('/')[0]
switch_s1_gw = switch_s1_iface_ip.split('/')[0]
switch_s2_gw = switch_s2_iface_ip.split('/')[0]

print(switch_client_gw)
print(switch_s1_gw)
print(switch_s2_gw)




192.168.1.1
192.168.2.1
192.168.3.1


In [22]:
stdout, stderr = client.execute(f'sudo ip route add {server2_switch_subnet} via {switch_client_gw}')
stdout, stderr = client.execute(f'sudo ip route add {server1_switch_subnet} via {switch_client_gw}')

stdout, stderr = server1.execute(f'sudo ip route add {client_switch_subnet} via {switch_s1_gw}')
stdout, stderr = server2.execute(f'sudo ip route add {client_switch_subnet} via {switch_s2_gw}')

In [23]:
### hardcoding arp so that these packets are not sent from the nodes to the bmv switch
stdout, stderr = client.execute(f'sudo arp -s {switch_client_gw} {switch_client_iface_MAC}')
stdout, stderr = server1.execute(f'sudo arp -s {switch_s1_gw} {switch_s1_iface_MAC}')
stdout, stderr = server2.execute(f'sudo arp -s {switch_s2_gw} {switch_s2_iface_MAC}')

In [ ]:
#stdout, stderr = switch.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

In [25]:
stdout, stderr = client.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)

In [26]:
server1.upload_file('cpu_intensive_task.py', 'cpu_intensive_task.py')
server2.upload_file('cpu_intensive_task.py', 'cpu_intensive_task.py')

<SFTPAttributes: [ size=1061 uid=1000 gid=1000 mode=0o100664 atime=1713751449 mtime=1713751449 ]>

In [8]:
stdout, stderr = client.execute(f'sudo apt-get update && sudo apt install apache2-utils -y', quiet=True)
stdout, stderr1 = server1.execute(f"sudo apt update && sudo apt install python3 python3-pip -y", quiet=True)
stdout, stderr1 = server2.execute(f"sudo apt update && sudo apt install python3 python3-pip -y", quiet=True)
stdout, stderr1 = switch.execute(f"sudo apt update && sudo apt install python3 python3-pip -y", quiet=True)
stdout, stderr1 = server1.execute(f"sudo apt update && yes | sudo pip3 install flask asyncio 'flask[async]'", quiet=True)
stdout, stderr2 = server2.execute(f"sudo apt update && yes | sudo pip3 install flask asyncio 'flask[async]'", quiet=True)
stdout, stderr2 = switch.execute(f"sudo apt update && yes | pip3 install flask", quiet=True)



In [10]:
switch.upload_file('install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

In [25]:
### high performance switching
switch.upload_file('scripts/simple_switch_hp.gz', '/home/ubuntu/simple_switch_hp.gz')
switch.execute('gzip -d simple_switch_hp.gz')
switch.execute('chmod +x /home/ubuntu/simple_switch_hp')
stdout, stderr = switch.execute(f'sudo pkill switch')
switch.upload_file('tcp-checksum-port-mapping.p4', 'main.p4')

switch.upload_file('scripts/disable_offload.sh', 'disable_offload.sh')
stdout, stderr = switch.execute(f'sudo chmod +x ./disable_offload.sh && sudo ./disable_offload.sh {switch_client_iface_name}', quiet=True)
stdout, stderr = switch.execute(f'sudo chmod +x ./disable_offload.sh && sudo ./disable_offload.sh {switch_s1_iface_name}', quiet=True)
stdout, stderr = switch.execute(f'sudo chmod +x ./disable_offload.sh && sudo ./disable_offload.sh {switch_s2_iface_name}', quiet=True)
### run the hp switch
print(f'sudo /home/ubuntu/simple_switch_hp -i 0@{switch_client_iface_name} -i 1@{switch_s1_iface_name} -i 2@{switch_s2_iface_name} main.json &')

 gzip: simple_switch_hp already exists;	not overwritten
 sudo /home/ubuntu/simple_switch_hp -i 0@enp8s0 -i 1@enp7s0 -i 2@enp10s0 main.json &


In [26]:
#upload match action rules directly from the rules file
switch.upload_file('tcp-checksum-latest-rules.sh', 'tcp-checksum-latest-rules.sh')
stdout, stderr = switch.execute('chmod +x tcp-checksum-latest-rules.sh && ./tcp-checksum-latest-rules.sh')


Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to exact match table MyIngress.forward_nat
match key:           EXACT-00	EXACT-00:00:00:00	EXACT-00:00
action:              MyIngress.send_to_server
runtime data:        00:00:00:00:00:03	00:00:00:00:00:04	c0:a8:02:0a	00:01	00:50
Entry has been added with handle 0
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to exact match table MyIngress.forward_nat
match key:           EXACT-01	EXACT-00:00:00:00	EXACT-00:00
action:              MyIngress.send_to_server
runtime data:        00:00:00:00:00:03	00:00:00:00:00:04	c0:a8:02:0a	00:01	00:50
Entry has been added with handle 1
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to exact match table MyIngress.forward_nat
match key:           EXACT-01	EXACT-00:00:00:01	EXACT-00:00
action:    

### change register state to 0


In [ ]:

switch.upload_file('register-modify-state0.sh', 'register-modify-state0.sh')
stdout, stderr = switch.execute('chmod +x register-modify-state0.sh && ./register-modify-state0.sh')


### change register state to 1

In [25]:

switch.upload_file('register-modify-state1.sh', 'register-modify-state1.sh')
stdout, stderr = switch.execute('chmod +x register-modify-state1.sh && ./register-modify-state1.sh')


Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: RuntimeCmd: 


In [ ]:
server1.upload_file('cpu_intensive_task.py', 'cpu_intensive_task.py')
server2.upload_file('cpu_intensive_task.py', 'cpu_intensive_task.py')

### high perfm switching and disable tcp offloading

### we now start setting up of the mflib on the nodes


In [21]:
# Check version
import mflib 
print(f"MFLib version  {mflib.__version__} " )

# Import MFLib Class
from mflib.mflib import MFLib

MFLib version  1.0.4 


In [22]:
%%time
mf = MFLib(slice_name)
instrumetize_results = mf.instrumentize( ["prometheus"] )

Inititializing slice "kalhan-cse-534-nat-2-servers-minimal-servers" for MeasurementFramework.
Found meas node as meas-node at 2620:0:c80:1003:f816:3eff:fe38:7a81
Bootstrap status indicates Slice Measurement Framework is ready.
Instrumentizing slice "kalhan-cse-534-nat-2-servers-minimal-servers"
   Setting up Prometheus...
   Setting up Prometheus done.
   Setting up grafana_manager & dashboards...
   Setting up grafana_manager & dashboards done.
Instrumentize Process Complete.
CPU times: user 2.47 s, sys: 66.3 ms, total: 2.53 s
Wall time: 1min


In [23]:
# Grafana SSH Tunnel Command
# mf.grafana_tunnel_local_port = 10010 # optionally change the port
print(mf.grafana_tunnel)

print(f"Browse to https://localhost:{mf.grafana_tunnel_local_port}/grafana/dashboards?query=%2A")

ssh -L 10010:localhost:443 -F ssh_config -i slice_key ubuntu@2620:0:c80:1003:f816:3eff:fe38:7a81
Browse to https://localhost:10010/grafana/dashboards?query=%2A


In [24]:
#### CREDENTIALS FOR ACCESSING THE PROMETHUES API

from mflib.data_transfer import PrometheusExporter
prom_tools = PrometheusExporter(slice_name = slice_name)
data={}
data["get"] = ["ht_user", "ht_password"]
prom_creds = prom_tools.info("prometheus",data)
print(prom_creds)

Inititializing slice "kalhan-cse-534-nat-2-servers-minimal-servers" for MeasurementFramework.
Found meas node as meas-node at 2620:0:c80:1003:f816:3eff:fe38:7a81
Bootstrap status indicates Slice Measurement Framework is ready.
{'ht_user': 'QrVrlLPR', 'ht_password': 'UZCMLftt'}


#### uploading server monitoring code to meas node


In [28]:
meas_node.upload_file("server_load_monitoring.py", "server_load_monitoring.py")
# server1.os_reboot()

<SFTPAttributes: [ size=2000 uid=1000 gid=1000 mode=0o100664 atime=1714015736 mtime=1714015765 ]>

In [19]:
switch.upload_file("switch_webhook.py", "switch_webhook.py")

<SFTPAttributes: [ size=1667 uid=1000 gid=1000 mode=0o100664 atime=1714013796 mtime=1714013963 ]>

In [ ]:
### Install scapy on client and server1
stdout, stderr = client.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)
stdout, stderr = server1.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)


In [ ]:
server1.upload_file('jupyter-examples-rel1.6.1/fabric_examples/complex_recipes/p4_labs_bmv2/scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = server1.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

In [ ]:
switch.upload_file('jupyter-examples-rel1.6.1/fabric_examples/complex_recipes/p4_labs_bmv2/labs_files/cse534/rules-with-nat-del.sh', 'rules-with-nat-del.sh')

In [ ]:
stdout, stderr = server2.execute(f'sudo python3 send.py enp7s0 192.168.3.10 192.168.3.1 hello')

In [ ]:


l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [
    (n.get_name(), {'color': 'red' if 'client' in n.get_name() else 'blue' if 'server' in n.get_name() else 'green' if n.get_name() == 'meas-node' else 'lightblue'})
    for n in slice.get_nodes()
]


nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]
     
     

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=[],
                             font_color='gray',  font_size=8, rotate=False);